In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../Banco de Súmulas - Sumulas.csv')
campeonatos = pd.read_csv('../Banco de Súmulas - Campeonatos.csv')

# **> Ajuste do ID da coluna `ID Campeonato`**

In [3]:
def fixIDCampeonato(df, campeonatos):
    """
    Função que ajusta o ID do campeonato de acordo com o nome do campeonato.
    Recebe como parâmetro o dataframe e o dataframe de campeonatos.
    """
    # turn Campeonatos into a dictionary
    campeonatos_dict = {}
    for index, row in campeonatos.iterrows():
        campeonatos_dict[row['Nome Competição']] = row['ID Competição']
    
    # fix ID Campeonato
    for index, row in df.iterrows():
        campeonato = row['Nome Competição']
        try:
            df.at[index, 'ID Competição'] = campeonatos_dict[campeonato]
        except:
            pass
    
    return df

In [4]:
df = fixIDCampeonato(df, campeonatos)

# **> Ajuste do formato da coluna `Pontuação Regata`**

In [5]:
df.head()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
0,NaN,NaN,Alejandro Cloos,2.0,Ilca 7,24,NaN,GERAL,19,NaN,205,174,Central and South American Championship 2022
1,NaN,NaN,Alejandro Cloos,2.0,Ilca 7,18,NaN,GERAL,19,NaN,205,174,Central and South American Championship 2022
2,NaN,NaN,Alejandro Cloos,2.0,Ilca 7,20,NaN,GERAL,19,NaN,205,174,Central and South American Championship 2022
3,NaN,NaN,Alejandro Cloos,2.0,Ilca 7,22,NaN,GERAL,19,NaN,205,174,Central and South American Championship 2022
4,NaN,NaN,Alejandro Cloos,2.0,Ilca 7,24,NaN,GERAL,19,NaN,205,174,Central and South American Championship 2022


**Casos de exemplo para tratamento:**

[x] 1.0

[x] 2.8

[x] 1

[x] -1

[x] (9.0)

[x] (9)

[x] (32.0 DNC)

[x] DSQ(69)

[x] DNF (32)

[x] (59.0 I BFD)

[x] (54.0 FL-SCP)

[x] DNF

[x] dnf lower case

[x] DNF 32

[x] RDG24

[x] RDG11.75

[x] 24.0 M-SCP

[x] 30.0 ARB

[x] 5.8 RDG F

[x] 4*BFD

[x] *18

In [6]:
def remove_parenthesis_or_asterisk(x):
    """
    Função que remove os parênteses e asteriscos de uma string.
    """
    if x.__contains__('('):
        return x.replace('(', '').replace(')', '')
    elif x.__contains__('*'):
        return x.replace('*', '')

def is_number(x):
    """
    Função que verifica se uma string é um número.
    """
    try:
        float(x)
        return True
    except:
        return False
    
def update(column, current_value, new_value, df, position):
    """
    Função que atualiza o valor de uma célula de um dataframe.
    Recebe como parâmetro o nome da coluna, o valor atual, o novo valor, o dataframe e a posição.
    Se o valor atual for None, o novo valor é atribuído à célula. Caso contrário, o valor atual é mantido.
    """
    if current_value == None:
        df.at[position, column] = new_value
    else:
        df.at[position, column] = current_value

def fixPontuacaoRegata(df):
    """
    Função que ajusta o formato da pontuação da regata considerando todos os casos específicos.
    Recebe como parâmetro o dataframe.
    """
    # define a lista de punicoes
    PUNICOES = ['I BFD', 'FL-SCP', 'M-SCP', 'DNC', 'DNS', 'PRP',
            'OCS', 'ZFP', 'UFD', 'BFD', 'SCP', 'NSC', 'DNF', 'SPI',
            'RET', 'DSQ', 'DNE', 'RDG', 'DPI', 'ARB', 'STP', 'RAF',
            'i bfd', 'fl-scp', 'm-scp', 'dnc', 'dns', 'spi', 'raf',
            'ocs', 'zfp', 'ufd', 'bfd', 'scp', 'nsc', 'dnf', 
            'ret', 'dsq', 'dne', 'rdg', 'dpi', 'arb', 'stp', 'prp']
    
    df = df.dropna(subset=['Pontuação Regata'])
    df['Descarte'] = df['Descarte'].replace(np.NaN, None)
    df['Punição'] = df['Punição'].replace(np.NaN, None)
    
    for i, row in df.iterrows():
        pontos = row['Pontuação Regata']
        descarte = row['Descarte']
        punicao = row['Punição']
        
        if pontos.startswith('-'):
            """
            Caso do bug do cntrl+c e cntrl+v, em que o número entre parênteses se tornou um número negativo
            """
            pontos = pontos[1:]

        if is_number(pontos):
            """
            Se pontos for um número, i.e., não tiver qualquer outro texto associado à string.
            Na coluna de Descarte, devemos adicionar 0 se o valor atual for np.nan.
            Na coluna de Punição, devemos manter o valor atual.
            """
            pontos = float(pontos)
            # change in the dataset itself
            df.at[i, 'Pontuação Regata'] = pontos
            update(column='Descarte', current_value=descarte, new_value=0, df=df, position=i)
            #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

        elif pontos.__contains__('(') or pontos.__contains__('*'):
            """
            Se pontos for uma string que contém um parênteses, pode ser um dos casos abaixo:
            (9.0), (32.0 DNC), DNF (32), DSQ(69).
            Nesse caso, se a remoção dos parênteses resultar em um número, devemos adicionar esse número
            na coluna de pontos. Se não, teremos que passar por mais alguns tratamentos de string.
            """
            pontos = remove_parenthesis_or_asterisk(pontos)
            
            if is_number(pontos):
                """
                Caso em que a remoção dos parênteses resulta apenas em um número.
                Nesse caso, houve descarte mas a indicação da punição não está na string ou não existe.
                Adicionamos 1 na coluna de Descarte, mas mantemos o valor atual na coluna de Punição.
                """
                pontos = float(pontos)
                df.at[i, 'Pontuação Regata'] = pontos
                update(column='Descarte', current_value=descarte, new_value=1, df=df, position=i)
                #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

            else:
                """
                Casos em que a remoção dos parênteses não resulta em um número puro.
                """
                for p in PUNICOES:
                    if pontos.__contains__(p):
                        """
                        Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                        Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                        a pontuação na coluna de pontos. Adicionamos 1 na coluna de Descarte e adicionamos a punição
                        na coluna de Punição.
                        """
                        pontos = pontos.replace(p, '')
                        try:
                            pontos = float(pontos)
                            df.at[i, 'Pontuação Regata'] = pontos
                        except:
                            df.at[i, 'Pontuação Regata'] = 'flag'
                        update(column='Descarte', current_value=descarte, new_value=1, df=df, position=i)
                        update(column='Punição', current_value=punicao, new_value=p.upper(), df=df, position=i)
                        break
            
        else:
            """
            Casos em que pontos não é um número e não possui parênteses (não há descarte), 
            mas há uma punição na string.
            """
            for p in PUNICOES:
                if pontos.__contains__(p):
                    """
                    Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                    Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                    a pontuação na coluna de pontos. Adicionamos 0 na coluna de Descarte e adicionamos a punição
                    na coluna de Punição.
                    """
                    pontos = pontos.replace(p, '')
                    try:
                        pontos = float(pontos)
                        df.at[i, 'Pontuação Regata'] = pontos
                    except:
                        df.at[i, 'Pontuação Regata'] = 'flag'
                    update(column='Descarte', current_value=descarte, new_value=0, df=df, position=i)
                    update(column='Punição', current_value=punicao, new_value=p.upper(), df=df, position=i)
                    break
    
    return df

In [7]:
df_ajustado = fixPontuacaoRegata(df)

In [8]:
df_ajustado.to_excel('ajustado.xlsx')

In [9]:
# # join df and df
# df = df_ajustado.join(df[['Pontuação Regata', 'Descarte', 'Punição']], rsuffix='_ajustada')

# # filter columns
# columns = ['Pontuação Regata', 'Descarte', 'Punição', 'Pontuação Regata_ajustada', 'Descarte_ajustada', 'Punição_ajustada']
# df = df[columns]

# # sample rows
# df.sample(20000)

# df.to_excel('pontuacao_ajustada.xlsx', index=False)

## **> Obtenção do dataset final**

In [10]:
# # colunas que vão entrar no dataframe final
# columns_to_model = ['Nome Competidor', 'Classe Vela', 'Flotilha', 'Posição Geral', 'Nett', 'Nome Competição']
# df_to_model = df[columns_to_model]

# # dropa as repetições regata a regata (mantém a primeira)
# df_to_model.drop_duplicates(keep='first', inplace=True)

# df_to_model.head()